In [1]:
import chart_studio.plotly as pl
import plotly.graph_objs as gobj
import pandas as pd
import plotly
import os
from datetime import date
from datetime import datetime
from datetime import timedelta
from urllib.parse import unquote
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
os.chdir(r'C:\Users\admin\Documents\Sourav\Code\RnD\Corona')


confirmed_df = pd.read_csv(unquote('https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_confirmed_global.csv&filename=time_series_covid19_confirmed_global.csv'))
# confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
del confirmed_df['Province/State']
confirmed_df.rename(columns={"Country/Region": "Country"},inplace=True)
confirmed_df = pd.melt(confirmed_df,id_vars=['Country','Lat','Long'], var_name='Day', value_name='ConfirmedCount')


recovered_df = pd.read_csv(unquote('https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_recovered_global.csv&filename=time_series_covid19_recovered_global.csv'))
# recovered_df = pd.read_csv('time_series_covid19_recovered_global.csv')
del recovered_df['Province/State']
recovered_df.rename(columns={"Country/Region": "Country"},inplace=True)
recovered_df = pd.melt(recovered_df,id_vars=['Country','Lat','Long'], var_name='Day', value_name='RecoveredCount')


death_df = pd.read_csv(unquote('https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_deaths_global.csv&filename=time_series_covid19_deaths_global.csv'))
# death_df = pd.read_csv('time_series_covid19_deaths_global.csv')
del death_df['Province/State']
death_df.rename(columns={"Country/Region": "Country"},inplace=True)
death_df = pd.melt(death_df,id_vars=['Country','Lat','Long'], var_name='Day', value_name='DeathCount')

finaldf = confirmed_df.merge(recovered_df,on=["Country",'Lat','Long','Day'],how='left').merge(death_df,on=["Country",'Lat','Long','Day'],how='left')

del finaldf['Lat']
del finaldf['Long']

finaldf['Day'] = pd.to_datetime(finaldf['Day'])
group_df = finaldf.groupby(['Country','Day'])['ConfirmedCount','RecoveredCount','DeathCount'].sum().reset_index().sort_values('Country')
# group_df = group_df[group_df['Day'] == '2020-04-14']
# group_df['RecoveredCount'] = group_df['RecoveredCount'].astype(int)
# canada_df = group_df.loc[group_df['Country']=='Canada']

# Subtracting 1 day from current date---
dateTimeObj = datetime.now() - timedelta(days=1)

# Converting datetime type to date type---
# y = x.date()

timestampStr = dateTimeObj.strftime("%Y-%m-%d")

# for col in group_df.columns:
#     group_df[col] = group_df[col].astype(str)

group = group_df.loc[(group_df['Day'] == timestampStr)].reset_index()


group['text'] = 'Confirmed : '+ group['ConfirmedCount'].astype(str) + ','+ '<br>' + \
'Recovered : '+ group['RecoveredCount'].astype(str) + ','+ '<br>' + \
'As on : ' + group['Day'].astype(str)


data  = dict(type = 'choropleth',
              autocolorscale = False,
              locations = group['Country'].tolist(),
              z = group['DeathCount'].tolist(),
              locationmode = 'country names',
              text = group['text'].tolist(),
              colorscale = 'Rainbow',
              colorbar = {'title':'No of deaths','len':350,'lenmode':'pixels' })

#initializing the layout variable
layout = dict(title = 'Corona death analysis across the world as on ' + timestampStr, geo = {'scope':'world'})

# Initializing the Figure object by passing data and layout as arguments.
col_map = gobj.Figure(data = [data],layout = layout)
plotly.offline.plot(col_map, filename='Covid-19_map.html')
# pl.iplot(col_map, filename = 'Choropleth-map.html')
# iplot(col_map)

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:41: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



'Covid-19_map.html'

In [ ]:
# type : ‘choropleth’ specifies that we are plotting a choropleth map.
# locations : The names of countries we want to plot.
# locationmode : It specifies that the plotting level is country wise. The value can be one of 3,-“ISO-3” , “USA-states” , “country names”.
# colorscale : The colour set used to plot the map.Available color scales are ‘Greys’, ‘YlGnBu’, ‘Greens’, ‘YlOrRd’, ‘Bluered’, ‘RdBu’, ‘Reds’, ‘Blues’, ‘Picnic’, ‘Rainbow’, ‘Portland’, ‘Jet’, ‘Hot’, ‘Blackbody’, ‘Earth’, ‘Electric’, ‘Viridis’, ‘Cividis’
# text: The textual information that needs to be displayed for each country on hover. 
# z: The value or factor that is used to distinguish the countries. These values are used by the colour scale.
# colorbar: A dictionary of parameters and arguments to customize the display of colorbar.Used to control the properties of the colorbar such as length, title, axis etc.
# geo: The parameter sets the properties of the map layout. The scope parameter sets the scope of the map. Scope can have any of the 7 values- “world” | “usa” | “europe” | “asia” | “africa” | “north america” | “south america” .